<a href="https://colab.research.google.com/github/Arvaloplz/Taller-Minimax-y-alpha-beta/blob/main/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimax y alpha-beta pruning para el juego del gato



<div>
<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Tic_tac_toe.svg/800px-Tic_tac_toe.svg.png" width="200"/>
</center>
</div>

Definimos estados y acciones para el juego.

Un estado (`State`) tendrá los siguientes atributos:

* **square**: matriz que representa el tablero de juego
* **next_player**: un carácter que representa al jugador que le toca jugar ('x', 'o') 
* **n_moves**: la cantidad de jugadas realizadas hasta el estado actual
* **value**: valor del estado actual (será calculado por el algoritmo de búsqueda). 

En cuanto a las acciones(`Action`) simplemente indicarán la casilla `(i,j)` correspondiente a la siguiente jugada.



In [ ]:
class State: 
    def __init__(self, square, n_moves=0, next_player='x', pos_last_move=None):    
        #state attributes
        self.square = deepcopy(square)
        self.next_player = next_player
        self.n_moves = n_moves
        self.pos_last_move = pos_last_move
        self.value = None
        self.marked= False
        self.children= []

 
    def get_actions(self):
        actions = []
        for i in range(3):
          for j in range(3):
            if self.square[i][j] == '-':
              actions.append(Action(i,j))
        return actions
 
    def transition(self, action):
        self.pos_last_move = (action.i,action.j)
        self.square[action.i][action.j] = self.next_player
        if self.next_player == 'x': self.next_player='o'
        else: self.next_player = 'x'
        self.n_moves += 1
 
    # retorna al ganador de la partida (en caso de ser un estado final)
    def get_winner(self):
        if self.pos_last_move is None: return None
        last_i, last_j = self.pos_last_move
        symbol = self.square[last_i][last_j]
        for case in range(4):
          final = True

          if case==0:
            for j in range(0,3): #horizontal --
              if symbol != self.square[last_i][j]: final=False
          elif case==1:
            for i in range(0,3): #vertival |
              if symbol != self.square[i][last_j]: final=False
          elif case==2 and last_i==last_j: #diagonal \
            for i in range(0,3):
                j=i
                if symbol != self.square[i][j]: final=False
          elif case==3 and last_j==2-last_i: #diagonal /
            for i in range(0,3):
                j=2-i
                if symbol != self.square[i][j]: final=False
          else: final=False

            
          if final: return symbol
        return None
 
    def __copy__(self):
        return type(self)(self.square, self.n_moves, self.next_player, self.pos_last_move)

In [ ]:
import numpy as np
from copy import copy, deepcopy
 
class Action:
    def __init__(self, i, j):
        self.i=i
        self.j=j
 
    def __str__(self):
        return str(self.i)+","+str(self.j)

## Ejemplo para probar la representación

In [ ]:
square = [['-','-','-'],
          ['-','-','-'],
          ['-','-','-']]

s=State(square)
s.transition(Action(1,1))
s.transition(Action(0,0))
s.transition(Action(0,1))
s.transition(Action(0,2))
s.transition(Action(2,1))
 
for line in s.square: print(line)

print("\nthe winner is", s.get_winner())

['o', 'x', 'o']
['-', 'x', '-']
['-', 'x', '-']

the winner is x


## Minimax

Ahora implementaremos el **minimax**.

El algoritmo minimax retorna la acción que genera el estado con un máximo **valor**

Para calcular el valor de un estado:

- Si le toca al jugador, el valor será igual al máximo valor de los estados hijos.

- Si le toca al oponente, el valor será igual al mínimo valor de los estados hijos.

Como se puede apreciar, la definción es **recursiva**, para calcular el valor de los estados de manera iterativa, realizaremos un recorrido post-order del árbol de búsqueda, es decir, comenzaremos por las hijos y luego pasaremos a los nodos padre (como en la figura).

![image](https://i.imgur.com/MHRnqFH.png)

En el código, observe que para realizar el post-order, necesitamos pasar dos veces por cada nodo, la **primera vez** lo marcamos y agregamos sus hijos (no se elimina el nodo). La **segunda pasada** se calcula el valor en función del valor de los hijos (min o max).


In [ ]:
def iterative_minimax(state):
  state = deepcopy(state)
  jugador = state.next_player
  stack = [state]

  iter_counter = 0
  while len(stack) > 0:
    s = stack[-1] # no se elimina del stack

    if s.marked == False: # primera visita
      s.marked = True

      if s.get_winner() is not None: continue # final state

      iter_counter+=1
      actions = s.get_actions()

      for a in actions:
        child = copy(s) # no copia los hijos del nodo
        child.transition(a)
        stack.append(child)

        # se agrega el hijo a s
        s.children.append(child)

    else: # en la segunda visita calculamos el valor
      if len(s.children) == 0: #final state
        #en caso de ser estado final asignamos 
        #1 si gana el jugador, -1 si pierde y 0 si empatan
        winner = s.get_winner()
        s.value=0
        if jugador == winner:
          s.value=1
        elif winner is not None:
          s.value=-1

      else: #not final state
         # el valor se obtiene de los estados hijos
         if jugador == s.next_player:
           s.value = max([ss.value for ss in s.children])
         else:
           s.value = min([ss.value for ss in s.children])
      stack.pop()

  #retorna la lista de acciones con sus valores asociados
  return [[ss.pos_last_move, ss.value] for ss in state.children], iter_counter
     

Ahora realizaremos una prueba. Inicalizamos un gato con ventaja para el jugador 'x'. 

In [ ]:
square = [['-','-','-'],
          ['-','-','-'],
          ['-','-','-']]

s=State(square)
s.transition(Action(1,1))
s.transition(Action(0,0))
s.transition(Action(0,1))
for line in s.square: print(line)

['o', 'x', '-']
['-', 'x', '-']
['-', '-', '-']


Tendrá opciones de empatar o ganar el jugador 'o' ?

Veamos.

In [ ]:
actions, iters = iterative_minimax(s)

print("jugador:",s.next_player)
print("valores para cada acción:", actions)
print("iterations:", iters)

jugador: o
valores para cada acción: [[(0, 2), -1], [(1, 0), -1], [(1, 2), -1], [(2, 0), -1], [(2, 1), 0], [(2, 2), -1]]
iterations: 690


Al lado de cada acción (del estado inicial) podemos ver el valor asociado al nodo hijo. Existe una acción (2,1) en la cuál el jugador 'o' tiene posibilidades de empatar.

Realicémosla!

In [ ]:
s.transition(Action(2,1))
for line in s.square: print(line)

['o', 'x', '-']
['-', 'x', '-']
['-', 'o', '-']


Haré una jugada "mala" de manera manual:

In [ ]:
s.transition(Action(0,2))  # <--- jugada mala
for line in s.square: print(line)

['o', 'x', 'x']
['-', 'x', '-']
['-', 'o', '-']


Veamos qué nos dice el minimax

In [ ]:
actions, iters = iterative_minimax(s)

print("jugador:",s.next_player)
print("valores para cada acción:", actions)
print("iterations:", iters)

jugador: o
valores para cada acción: [[(1, 0), -1], [(1, 2), -1], [(2, 0), 1], [(2, 2), -1]]
iterations: 28


Según minimax, si jugamos (2,0) ganaremos la partida!

In [ ]:
s.transition(Action(2,0))
for line in s.square: print(line)

['o', 'x', 'x']
['-', 'x', '-']
['o', 'o', '-']


Lo ven?, el jugador 'x' ya no tiene posibilidades y se retira.

## Poda alpha beta

Para implementar la poda alpha-beta, incluiremos dos atributos a cada nodo:
* $\alpha$: una **cota inferior** para el máximo valor de los hijos (max >= $\alpha$).
* $\beta$: una **cota superior** para el mínimo valor de los hijos  (min <= $\beta$).

Los hijos heredan los valores $\alpha$ y $\beta$ de sus padres.

Cada vez que calculemos el valor de un nodo, revisaremos y actualizaremos los valores de $\alpha$ o $\beta$ del nodo padre. Los caso son los siguientes:

* Si el padre es un nodo max, y el valor del nodo es mayor a $\alpha$, actualizamos ($\alpha=$ valor del nodo).

* Si el padre es un nodo min, y el valor del nodo es menor a $\beta$, actualizamos ($\beta=$ valor del nodo).

Además, cuando el valor $\beta$ de un nodo sea menor o igual al $\alpha$ de su padre, podremos descartar inmediatamente la rama. Lo mismo ocurre si el valor $\alpha$ de un nodo es mayor o igual al $\beta$ del padre.

In [ ]:
def iterative_alpha_beta(state):
  state = deepcopy(state)
  state.alpha = -np.inf
  state.beta = np.inf
  state.parent=None

  jugador = state.next_player
  stack = [state]

  iter_counter = 0
  while len(stack) > 0:
    s = stack[-1] # no se elimina del stack

    if s.marked == False: # primera visita
      s.marked = True

      #descarte de rama
      if s.parent is not None and s.parent.parent is not None:
        if jugador == s.next_player and s.parent.beta <= s.parent.parent.alpha: continue
        if jugador != s.next_player and s.parent.alpha >= s.parent.parent.beta: continue

      if s.get_winner() is not None: continue # final state

      iter_counter+=1

      actions = s.get_actions()

      for a in actions:
        child = copy(s) # no copia los hijos del nodo (copia valores de alpha y beta)
        child.alpha=s.alpha; child.beta=s.beta
        child.parent=s

        child.transition(a)
        stack.append(child)

        # se agrega el hijo a s
        s.children.append(child)

    else: # en la segunda visita calculamos el valor
      if len(s.children) == 0: #final state
        #en caso de ser estado final asignamos 
        #1 si gana el jugador, -1 si pierde y 0 si empatan
        winner = s.get_winner()
        s.value=0
        if jugador == winner:
          s.value=1
        elif winner is not None:
          s.value=-1

      else: #not final state
         # el valor se obtiene de los estados hijos
         if jugador == s.next_player:
           s.value = max([ss.value for ss in s.children])
         else:
           s.value = min([ss.value for ss in s.children])

      if jugador == s.next_player:
        if s.parent is not None and s.value < s.parent.beta: s.parent.beta=s.value
      else:
        if s.parent is not None and s.value > s.parent.alpha: s.parent.alpha=s.value

      stack.pop()

  #retorna la lista de acciones con sus valores asociados
  return [[ss.pos_last_move, ss.value] for ss in state.children], iter_counter
     

Hagamos unas pruebas con el tablero vacío.

In [ ]:
square = [['-','-','-'],
          ['-','-','-'],
          ['-','-','-']]

s=State(square)
for line in s.square: print(line)

['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


In [ ]:
actions, iters = iterative_alpha_beta(s)

print("jugador:",s.next_player)
print("valores para cada acción:", actions)
print("iterations:", iters)

jugador: x
valores para cada acción: [[(0, 0), 0], [(0, 1), 0], [(0, 2), 0], [(1, 0), 0], [(1, 1), 0], [(1, 2), 0], [(2, 0), 0], [(2, 1), 0], [(2, 2), 0]]
iterations: 19327


En sólo 19327 iteraciones obtenemos los valores de las acciones.

In [ ]:
square = [['-','-','-'],
          ['-','-','-'],
          ['-','-','-']]

s=State(square)
for line in s.square: print(line)

actions, iters = iterative_minimax(s)

print("jugador:",s.next_player)
print("valores para cada acción:", actions)
print("iterations:", iters)

['-', '-', '-']
['-', '-', '-']
['-', '-', '-']
jugador: x
valores para cada acción: [[(0, 0), 0], [(0, 1), 0], [(0, 2), 0], [(1, 0), 0], [(1, 1), 0], [(1, 2), 0], [(2, 0), 0], [(2, 1), 0], [(2, 2), 0]]
iterations: 340858


Comparadas con las 340858 de minimax!

# Actividad

Implemente un juego en python usando grafos (estados y acciones).

Aplique los algoritmos minimax y alpha-beta para ver que ocurre.

Algunas ideas:

- [Puzzles de ajedrez](https://chesspuzzle.net/)
- Juego de los 15 palitos ([Nim](https://es.wikipedia.org/wiki/Nim_(juego)#The_subtraction_game))
- Damas con tablero reducido
- [Go](https://es.wikipedia.org/wiki/Go)
- [Conecta 4](https://es.wikipedia.org/wiki/Conecta_4)
- [Aquí](https://en.wikipedia.org/wiki/Solved_game) pueden ver más juegos

No es necesario llegar hasta un estado final, puede *inventar una heurística* para evaluar estados no finales y limitar la altura del árbol.

Por ejemplo, cantidad de piezas o puntaje de piezas (damas o ajedrez), cantidad de piezas en linea (conecta 4), etc... 
